In [8]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from NFconstants import N_nod,a, N_traj
from andist import A, C
from tqdm import tqdm
import random
from torch.nn.utils.parametrizations import orthogonal

class L(nn.Linear):
    def __init__(self,n):
        super().__init__(n*(n+1)//2,1,bias=False)
        self.n=n        
    
    def anti_flatten(self):
        n=self.n
        w=self.weight
        Lw=torch.zeros((n,n))
        for i in range(n):
            for j in range(i+1):
                Lw[i][j]=w[0][i*(i+1)//2+j]  #лекция 14 
        return Lw        
        
    def det(self):
        det=1
        for i in range(self.n):
            det*=self.Lw[i][i]
    def log_abs_det(self):
        lad=0
        Lw=self.anti_flatten()
        for i in range(self.n):
            lad+=torch.log(torch.abs(Lw[i][i]))
        return lad
    def forward(self,x):
        Lwt=torch.t(self.anti_flatten())
        return torch.matmul(x,Lwt)
        
    def adj(self,mat):
        Lw=self.anti_flatten()
        Lwt=torch.t(Lw)
        D=torch.matmul(Lwt,torch.matmul(mat,Lw))
        return D
    


In [9]:
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from NFconstants import N_nod,a, N_traj
from andist import A, C
from tqdm import tqdm
import random
from Data import train_loader
from LOSS import KL_osc

def set_random_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)



A=torch.tensor(A).float()
C=torch.tensor(C).float()


class Action(nn.Module):
    def __init__(self,A):
        super().__init__()
        self.A=A
    
    def forward(self,x):
        y=torch.matmul(self.A,x)
        return 1/2 * torch.dot(y,x) 

class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x, act, model):
        loss=0
        M=len(x)
        for i in range(M):
            loss+=act(x[i])
        loss=loss/M
        lad=model.log_abs_det()
        loss=loss-lad
        return loss


def A_I(model):
    A_D=model.adj(A)
    I=torch.eye(N_nod)
    return torch.linalg.matrix_norm(A_D-I)
     
 
set_random_seed(42)    
model=L(N_nod)
#print(model.weight)
optimizer = torch.optim.Adam(model.parameters(), lr=0.00005)

act=Action(A)
loss_fn = KL_osc
num_epochs=1000
metric2=[A_I(model)]


for epoch in tqdm(range(num_epochs)):
    for x in train_loader:
        # Forward pass
        outputs = model(x)
        loss = loss_fn(outputs,-model.log_abs_det())
        

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    metric2.append(A_I(model))


fig=plt.figure()
plt.plot(list(range(num_epochs+1)),metric2)
plt.grid(True)
plt.show()


  2%|█▊                                                                              | 23/1000 [00:49<35:13,  2.16s/it]


KeyboardInterrupt: 